## Module 2: Basic Mathematical Methods for Forecasting

We build forecasting intuition using simple models as baselines.

### 2.1 Naive Forecasting Methods

In [1]:
# Load
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv")
df.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


#### 2.1.1 Mean Forecast

In [2]:
# Predict average of history
mean_value = df['Passengers'].mean()
df['MeanForecast'] = mean_value
df[['Passengers', 'MeanForecast']].head()

,Passengers,MeanForecast
0,112,280.298611
1,118,280.298611
2,132,280.298611
3,129,280.298611
4,121,280.298611


#### 2.1.2 Last Value (Naive) Forecast

In [3]:
df['NaiveForecast'] = df['Passengers'].shift(1)
df[['Passengers', 'NaiveForecast']].head()

,Passengers,NaiveForecast
0,112,NaN
1,118,112.0
2,132,118.0
3,129,132.0
4,121,129.0


#### 2.1.3 Seasonal Naive Forecast

In [4]:
season_length = 12  # For monthly data
df['SeasonalNaiveForecast'] = df['Passengers'].shift(season_length)
df[['Passengers', 'SeasonalNaiveForecast']].head(season_length+2)

,Passengers,SeasonalNaiveForecast
0,112,NaN
1,118,NaN
2,132,NaN
3,129,NaN
4,121,NaN
5,135,NaN
6,148,NaN
7,148,NaN
8,136,NaN
9,119,NaN


### 2.2 Smoothing Techniques
#### Moving Average

In [5]:
window = 3
df['MAForecast'] = df['Passengers'].rolling(window=window).mean().shift(1)
df[['Passengers', 'MAForecast']].head(window+2)

,Passengers,MAForecast
0,112,NaN
1,118,NaN
2,132,NaN
3,129,120.666667
4,121,126.333333


#### Simple, Double, and Triple Exponential Smoothing

In [6]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

train = df['Passengers'][:-12]

# Simple (no trend/seasonality)
ses_model = SimpleExpSmoothing(train).fit(smoothing_level=0.5, optimized=False)
ses_forecast = ses_model.forecast(12)
print(ses_forecast)

132    405.681091
133    405.681091
134    405.681091
135    405.681091
136    405.681091
137    405.681091
138    405.681091
139    405.681091
140    405.681091
141    405.681091
142    405.681091
143    405.681091
dtype: float64


In [7]:
# Double (trend)
holt_model = ExponentialSmoothing(train, trend='add', seasonal=None).fit()
holt_forecast = holt_model.forecast(12)
print(holt_forecast)

132    407.236678
133    409.473357
134    411.710036
135    413.946714
136    416.183393
137    418.420072
138    420.656751
139    422.893429
140    425.130108
141    427.366787
142    429.603466
143    431.840145
dtype: float64


In [8]:
# Triple (trend + seasonality)
hw_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12).fit()
hw_forecast = hw_model.forecast(12)
print(hw_forecast)

132    415.452953
133    397.086057
134    457.467964
135    445.365434
136    466.183776
137    520.727227
138    592.419534
139    598.839949
140    498.825488
141    442.379289
142    396.511604
143    424.286368
dtype: float64


### 2.3 Regression-Based Forecasts
#### Linear Regression with Time as a Feature

In [9]:
from sklearn.linear_model import LinearRegression
import numpy as np

df = df.copy()
df['TimeIndex'] = np.arange(len(df))
X = df[['TimeIndex']]
y = df['Passengers']

reg = LinearRegression().fit(X, y)
df['LinRegForecast'] = reg.predict(X)
df[['Passengers', 'LinRegForecast']].head()

,Passengers,LinRegForecast
0,112,90.309962
1,118,92.967146
2,132,95.624330
3,129,98.281513
4,121,100.938697


### 2.4 Model Evaluation
Evaluate how well each method works by calculating errors.

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(df['Passengers'].iloc[1:], df['NaiveForecast'].iloc[1:])
rmse = np.sqrt(mean_squared_error(df['Passengers'].iloc[1:], df['NaiveForecast'].iloc[1:]))
print(f"Naive MAE: {mae:.2f}, RMSE: {rmse:.2f}")

Naive MAE: 25.86, RMSE: 33.71


### 2.5 Mini Project
1. Load a (your own or above) dataset
2. Compute mean, naive, moving average forecasts
3. Try smoothing and regression
4. Visualize (plot) actual vs forecasts
5. Compare MAE/RMSE

### 2.6 Next Up
Statistical time series models (ARIMA, SARIMA, Prophet...)